In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=3030f594300c07b3a7924c29426702daab187a8707bef0bd2ccb4c62c2ff456f
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [ ]:
import pyspark
from pyspark.sql import SparkSession, SQLContext
spark = SparkSession.builder.appName('Movie Lens Recommendation').getOrCreate()

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

In [ ]:
lines = spark.read.text("/content/drive/MyDrive/B Data/Minggu14/ratings.dat").rdd
parts = lines.map(lambda row: row.value.split("::"))
ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),
                                     rating=int(p[2]), timestamp=int(p[3])))
ratings = spark.createDataFrame(ratingsRDD)
(training, test) = ratings.randomSplit([0.8, 0.2])

# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
predictions.show()

import math
result = predictions.rdd.map(lambda row: row['prediction'] - row['rating']).map(lambda x: x*x).filter(lambda x: not math.isnan(x))
mse = result.reduce(lambda x,y: x+y)

+------+-------+------+---------+----------+
|userId|movieId|rating|timestamp|prediction|
+------+-------+------+---------+----------+
|     1|      1|     5|978824268|  4.349672|
|     1|     48|     5|978824351|  3.769574|
|     1|    150|     5|978301777| 4.2829885|
|     1|    260|     4|978300760|  4.264531|
|     1|    720|     3|978300760| 4.1760583|
|     1|    914|     3|978301968|  4.353015|
|     1|    919|     4|978301368| 4.6437397|
|     1|   1028|     5|978301777|  4.698875|
|     1|   1097|     4|978301953| 4.5666614|
|     1|   1836|     5|978300172|   3.65844|
|     1|   2804|     5|978300719| 4.4577384|
|     2|     95|     2|978300143| 2.7067063|
|     2|    165|     3|978300002|  3.484576|
|     2|    434|     2|978300174|  3.024331|
|     2|    647|     3|978299351| 3.7268271|
|     2|    780|     3|978299966|  3.916191|
|     2|   1090|     2|978298580|  3.797369|
|     2|   1103|     3|978298905| 3.9133482|
|     2|   1188|     4|978299620| 2.9161522|
|     2|  